In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, GRU
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, mean_squared_error, recall_score, precision_score, f1_score, mean_absolute_error, r2_score, roc_curve, accuracy_score
import matplotlib.pyplot as plt
import time

# Load the dataset
file_path = './TFM_UNIR/data/processed/BINANCE_BTCUSD_preprocessed.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
data.head()

,time,open,high,low,close,Go_Short,Go_Long
0,2022-04-20 02:00:00-05:00,41460.2,41633.7,41286.9,41334.2,0,0
1,2022-04-20 03:00:00-05:00,41334.2,41516.9,41303.3,41477.5,0,0
2,2022-04-20 04:00:00-05:00,41477.4,41721.1,41453.0,41537.7,0,0
3,2022-04-20 05:00:00-05:00,41537.6,41896.0,41447.7,41810.8,0,0
4,2022-04-20 06:00:00-05:00,41810.8,42188.0,41711.4,42089.0,0,0


## Preprocesamiento de datos

In [4]:
# Convert the 'time' column to datetime
data['time'] = pd.to_datetime(data['time'])

# Sort data by time
data = data.sort_values('time')

# Select relevant features for prediction
features = ['open', 'high', 'low', 'close']
target = 'close'

# Normalize the features
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])

# Create sequences for the LSTM model
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length][3])  # Utilizar solo el precio de cierre como objetivo
    return np.array(X), np.array(y)
        #y.append(data[i + seq_length])
    #return np.array(X), np.array(y)

seq_length = 10  # Example sequence length
X, y = create_sequences(data[features].values, seq_length)

# Split the data into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(14253, 10, 4) (3564, 10, 4) (14253,) (3564,)


## Construcción y entrenamiento de modelos

In [7]:
# Crear modelo CNN
def build_cnn(input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Entrenamiento del modelo CNN
cnn_model = build_cnn(X_train.shape[1:])
start_time = time.time()
history_cnn = cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
cnn_train_time = time.time() - start_time

Epoch 1/10


C:\Users\Jhon_\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.1091e-04 - mae: 0.0060 - val_loss: 2.2853e-05 - val_mae: 0.0034
Epoch 2/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.2230e-05 - mae: 0.0031 - val_loss: 1.3340e-05 - val_mae: 0.0023
Epoch 3/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.9987e-05 - mae: 0.0030 - val_loss: 9.5989e-06 - val_mae: 0.0019
Epoch 4/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.8063e-05 - mae: 0.0029 - val_loss: 3.6504e-05 - val_mae: 0.0049
Epoch 5/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.6472e-05 - mae: 0.0028 - val_loss: 1.2508e-05 - val_mae: 0.0026
Epoch 6/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.4235e-05 - mae: 0.0024 - val_loss: 1.0604e-05 - val_mae: 0.0024
Epoch 7/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.5212e-05 - mae: 0.0026 - val_loss: 7.5135e-06 - val_mae: 0.0017
Epoch 8/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.2958e-05 - mae: 0.0024 - val_loss: 1.8106e-05 - val_mae: 0.0035
Epo

In [9]:
# LSTM Model
def build_lstm(input_shape):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Entrenamiento del modelo LSTM
lstm_model = build_lstm(X_train.shape[1:])
start_time = time.time()
history_lstm = lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
lstm_train_time = time.time() - start_time

Epoch 1/10


C:\Users\Jhon_\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


357/357 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 0.0012 - mae: 0.0158 - val_loss: 2.2368e-05 - val_mae: 0.0028
Epoch 2/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 3.0245e-05 - mae: 0.0036 - val_loss: 2.2078e-05 - val_mae: 0.0028
Epoch 3/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 2.7928e-05 - mae: 0.0034 - val_loss: 5.9270e-05 - val_mae: 0.0063
Epoch 4/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 3.0866e-05 - mae: 0.0037 - val_loss: 3.9463e-05 - val_mae: 0.0048
Epoch 5/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 2.5096e-05 - mae: 0.0033 - val_loss: 4.9145e-05 - val_mae: 0.0057
Epoch 6/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 2.6637e-05 - mae: 0.0034 - val_loss: 1.7607e-05 - val_mae: 0.0028
Epoch 7/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 2.5783e-05 - mae: 0.0034 - val_loss: 1.5973e-05 - val_mae: 0.0026
Epoch 8/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 2.1616e-05 - mae: 0.0030 - val_loss: 2.0994e-04 - val_mae: 0.0135
Ep

In [11]:
# GRU Model
def build_gru(input_shape):
    model = Sequential()
    model.add(GRU(50, activation='relu', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse',metrics=['mae'])
    return model

# Entrenamiento del modelo GRU
gru_model = build_gru(X_train.shape[1:])
start_time = time.time()
history_gru = gru_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
gru_train_time = time.time() - start_time

Epoch 1/10


C:\Users\Jhon_\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0034 - mae: 0.0261 - val_loss: 1.1874e-05 - val_mae: 0.0023
Epoch 2/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.1696e-05 - mae: 0.0021 - val_loss: 1.0616e-05 - val_mae: 0.0024
Epoch 3/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.2521e-05 - mae: 0.0022 - val_loss: 1.7845e-05 - val_mae: 0.0036
Epoch 4/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.2252e-05 - mae: 0.0022 - val_loss: 7.6708e-06 - val_mae: 0.0017
Epoch 5/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.1022e-05 - mae: 0.0021 - val_loss: 8.4301e-06 - val_mae: 0.0018
Epoch 6/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 9.9980e-06 - mae: 0.0020 - val_loss: 7.7258e-06 - val_mae: 0.0019
Epoch 7/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.1148e-05 - mae: 0.0021 - val_loss: 1.7528e-05 - val_mae: 0.0037
Epoch 8/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.1005e-05 - mae: 0.0021 - val_loss: 7.2595e-06 - val_mae: 0.0018
Epoch 9

## Evaluacion de modelos

In [14]:
# Umbralizar las etiquetas verdaderas a binario
y_test_bin = (y_test > 0.5).astype("int32")

# Función para evaluar un modelo y calcular todas las métricas
def evaluate_model(model, X_test, y_test, y_test_bin):
    start_time = time.time()
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype("int32")
    end_time = time.time()
    metrics = {}
    
    metrics['accuracy'] = accuracy_score(y_test_bin, y_pred)
    metrics['recall'] = recall_score(y_test_bin, y_pred)
    metrics['precision'] = precision_score(y_test_bin, y_pred)
    metrics['f1_score'] = f1_score(y_test_bin, y_pred)
    metrics['mse'] = mean_squared_error(y_test, y_pred_proba)
    metrics['mae'] = mean_absolute_error(y_test, y_pred_proba)
    metrics['r2'] = r2_score(y_test, y_pred_proba)
    metrics['confusion_matrix'] = confusion_matrix(y_test_bin, y_pred)
    metrics['roc_auc'] = roc_auc_score(y_test_bin, y_pred_proba)
    metrics['training_time'] = end_time - start_time
    
    return metrics

# Evaluar modelos
metrics_cnn = evaluate_model(cnn_model, X_test, y_test, y_test_bin)
metrics_lstm = evaluate_model(lstm_model, X_test, y_test, y_test_bin)
metrics_gru = evaluate_model(gru_model, X_test, y_test, y_test_bin)

print("CNN Metrics:", metrics_cnn)
print("LSTM Metrics:", metrics_lstm)
print("GRU Metrics:", metrics_gru)

112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
CNN Metrics: {'accuracy': 0.9974747474747475, 'recall': 0.9972027972027973, 'precision': 0.9985994397759104, 'f1_score': 0.9979006298110568, 'mse': 5.2560375722700956e-05, 'mae': 0.004500594209189425, 'r2': 0.9986193902580318, 'confusion_matrix': array([[1416,    3],
       [   6, 2139]], dtype=int64), 'roc_auc': 0.9998705546274256, 'training_time': 0.44969677925109863}
LSTM Metrics: {'accuracy': 0.9941077441077442, 'recall': 0.9925407925407925, 'precision': 0.9976569821930646, 'f1_score': 0.9950923112876839, 'mse': 0.00021484941657579808, 'mae': 0.01084010632818198, 'r2': 0.994356524406415, 'confusion_matrix': array([[1414,    5],
       [  16, 2129]], dtype=int64), 'roc_auc': 0.9998094458982407, 'training_time': 1.0274767875671387}
GRU Metrics: {'accuracy': 0.9938271604938271, 'recall': 0.9911421911421912, 'precision': 0.9985908877407234, 'f1_score': 0.9948525970

## Creacion de Dashboard interactivo

In [62]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output

# Definición de la gama de colores personalizada para las barras
bar_colors = ['#A8DADC', '#457B9D', '#1D3557']
dimmed_color = '#D3D3D3'

# Definición de la gama de colores personalizada para la matriz de confusión
custom_colorscale = [
    [0, '#A8DADC'],
    [0.5, '#457B9D'],
    [1, '#1D3557']
]

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Comparación de Modelos de Redes Neuronales", style={'textAlign': 'center'}),

    # Dropdown para seleccionar el modelo
    html.Label("Seleccione un Modelo:"),
    dcc.Dropdown(
        id='model-dropdown',
        options=[
            {'label': 'CNN', 'value': 'CNN'},
            {'label': 'LSTM', 'value': 'LSTM'},
            {'label': 'GRU', 'value': 'GRU'}
        ],
        value='CNN'  # Valor por defecto
    ),

    # Métricas de Clasificación y Rendimiento
    dcc.Graph(id='classification_metrics_comparison'),

    # Métricas de Error (MSE, MAE) y R² en una sola fila
    html.Div([
        dcc.Graph(id='mse_metrics_comparison', style={'width': '33%', 'display': 'inline-block'}),
        dcc.Graph(id='mae_metrics_comparison', style={'width': '33%', 'display': 'inline-block'}),
        dcc.Graph(id='r2_metrics_comparison', style={'width': '33%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'flex-direction': 'row'}),

    # Matrices de Confusión en una sola fila
    html.Div([
        dcc.Graph(id='confusion_matrix_cnn', style={'width': '33%', 'display': 'inline-block'}),
        dcc.Graph(id='confusion_matrix_lstm', style={'width': '33%', 'display': 'inline-block'}),
        dcc.Graph(id='confusion_matrix_gru', style={'width': '33%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'flex-direction': 'row'}),

    # ROC-AUC y Tiempos de Entrenamiento en una sola fila
    html.Div([
        dcc.Graph(id='roc_auc', style={'width': '50%', 'display': 'inline-block'}),
        dcc.Graph(id='training_time', style={'width': '50%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'flex-direction': 'row'})
])

@app.callback(
    [
        Output('classification_metrics_comparison', 'figure'),
        Output('mse_metrics_comparison', 'figure'),
        Output('mae_metrics_comparison', 'figure'),
        Output('r2_metrics_comparison', 'figure'),
        Output('confusion_matrix_cnn', 'figure'),
        Output('confusion_matrix_lstm', 'figure'),
        Output('confusion_matrix_gru', 'figure'),
        Output('roc_auc', 'figure'),
        Output('training_time', 'figure')
    ],
    [Input('model-dropdown', 'value')]
)
def update_graphs(selected_model):
    # Dim all bars except the selected model
    colors_classification = [bar_colors[0] if selected_model == 'CNN' else dimmed_color,
                             bar_colors[1] if selected_model == 'LSTM' else dimmed_color,
                             bar_colors[2] if selected_model == 'GRU' else dimmed_color]

    figure_classification = {
        'data': [
            go.Bar(
                name='CNN', 
                x=['Accuracy', 'Recall', 'Precision', 'F1 Score', 'ROC-AUC'], 
                y=[metrics_cnn['accuracy'], metrics_cnn['recall'], metrics_cnn['precision'], metrics_cnn['f1_score'], metrics_cnn['roc_auc']], 
                width=0.2, 
                marker_color=colors_classification[0],
                text=[f'{metric:.2f}' for metric in [metrics_cnn['accuracy'], metrics_cnn['recall'], metrics_cnn['precision'], metrics_cnn['f1_score'], metrics_cnn['roc_auc']]],
                textposition='auto', 
                insidetextanchor='middle'
            ),
            go.Bar(
                name='LSTM', 
                x=['Accuracy', 'Recall', 'Precision', 'F1 Score', 'ROC-AUC'], 
                y=[metrics_lstm['accuracy'], metrics_lstm['recall'], metrics_lstm['precision'], metrics_lstm['f1_score'], metrics_lstm['roc_auc']], 
                width=0.2, 
                marker_color=colors_classification[1],
                text=[f'{metric:.2f}' for metric in [metrics_lstm['accuracy'], metrics_lstm['recall'], metrics_lstm['precision'], metrics_lstm['f1_score'], metrics_lstm['roc_auc']]],
                textposition='auto', 
                insidetextanchor='middle'
            ),
            go.Bar(
                name='GRU', 
                x=['Accuracy', 'Recall', 'Precision', 'F1 Score', 'ROC-AUC'], 
                y=[metrics_gru['accuracy'], metrics_gru['recall'], metrics_gru['precision'], metrics_gru['f1_score'], metrics_gru['roc_auc']], 
                width=0.2, 
                marker_color=colors_classification[2],
                text=[f'{metric:.2f}' for metric in [metrics_gru['accuracy'], metrics_gru['recall'], metrics_gru['precision'], metrics_gru['f1_score'], metrics_gru['roc_auc']]],
                textposition='auto', 
                insidetextanchor='middle'
            )
        ],
        'layout': go.Layout(
            title='Comparación de Métricas de Clasificación y Rendimiento',
            barmode='group',
            yaxis={'range': [0.8, 1.0]}  # Ajuste del rango del eje y
        )
    }

    figure_mse = {
        'data': [
            go.Bar(
                name='CNN', 
                x=['MSE'], 
                y=[metrics_cnn['mse']], 
                width=0.2, 
                marker_color=colors_classification[0],
                text=[f'{metrics_cnn["mse"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            ),
            go.Bar(
                name='LSTM', 
                x=['MSE'], 
                y=[metrics_lstm['mse']], 
                width=0.2, 
                marker_color=colors_classification[1],
                text=[f'{metrics_lstm["mse"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            ),
            go.Bar(
                name='GRU', 
                x=['MSE'], 
                y=[metrics_gru['mse']], 
                width=0.2, 
                marker_color=colors_classification[2],
                text=[f'{metrics_gru["mse"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            )
        ],
        'layout': go.Layout(title='Comparación de MSE', barmode='group')
    }

    figure_mae = {
        'data': [
            go.Bar(
                name='CNN', 
                x=['MAE'], 
                y=[metrics_cnn['mae']], 
                width=0.2, 
                marker_color=colors_classification[0],
                text=[f'{metrics_cnn["mae"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            ),
            go.Bar(
                name='LSTM', 
                x=['MAE'], 
                y=[metrics_lstm['mae']], 
                width=0.2, 
                marker_color=colors_classification[1],
                text=[f'{metrics_lstm["mae"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            ),
            go.Bar(
                name='GRU', 
                x=['MAE'], 
                y=[metrics_gru['mae']], 
                width=0.2, 
                marker_color=colors_classification[2],
                text=[f'{metrics_gru["mae"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            )
        ],
        'layout': go.Layout(title='Comparación de MAE', barmode='group')
    }

    figure_r2 = {
        'data': [
            go.Bar(
                name='CNN', 
                x=['R²'], 
                y=[metrics_cnn['r2']], 
                width=0.2, 
                marker_color=colors_classification[0],
                text=[f'{metrics_cnn["r2"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            ),
            go.Bar(
                name='LSTM', 
                x=['R²'], 
                y=[metrics_lstm['r2']], 
                width=0.2, 
                marker_color=colors_classification[1],
                text=[f'{metrics_lstm["r2"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            ),
            go.Bar(
                name='GRU', 
                x=['R²'], 
                y=[metrics_gru['r2']], 
                width=0.2, 
                marker_color=colors_classification[2],
                text=[f'{metrics_gru["r2"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            )
        ],
        'layout': go.Layout(title='Comparación de R²', barmode='group')
    }

    figure_roc_auc = {
        'data': [
            go.Bar(
                name='CNN', 
                x=['ROC-AUC'], 
                y=[metrics_cnn['roc_auc']], 
                width=0.2, 
                marker_color=colors_classification[0],
                text=[f'{metrics_cnn["roc_auc"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            ),
            go.Bar(
                name='LSTM', 
                x=['ROC-AUC'], 
                y=[metrics_lstm['roc_auc']], 
                width=0.2, 
                marker_color=colors_classification[1],
                text=[f'{metrics_lstm["roc_auc"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            ),
            go.Bar(
                name='GRU', 
                x=['ROC-AUC'], 
                y=[metrics_gru['roc_auc']], 
                width=0.2, 
                marker_color=colors_classification[2],
                text=[f'{metrics_gru["roc_auc"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            )
        ],
        'layout': go.Layout(title='ROC-AUC de los Modelos', barmode='group')
    }

    figure_training_time = {
        'data': [
            go.Bar(
                name='CNN', 
                x=['Training Time'], 
                y=[metrics_cnn['training_time']], 
                width=0.2, 
                marker_color=colors_classification[0],
                text=[f'{metrics_cnn["training_time"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            ),
            go.Bar(
                name='LSTM', 
                x=['Training Time'], 
                y=[metrics_lstm['training_time']], 
                width=0.2, 
                marker_color=colors_classification[1],
                text=[f'{metrics_lstm["training_time"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            ),
            go.Bar(
                name='GRU', 
                x=['Training Time'], 
                y=[metrics_gru['training_time']], 
                width=0.2, 
                marker_color=colors_classification[2],
                text=[f'{metrics_gru["training_time"]:.2f}'], 
                textposition='auto', 
                insidetextanchor='middle'
            )
        ],
        'layout': go.Layout(title='Tiempo de Entrenamiento de los Modelos', barmode='group')
    }

    figure_confusion_cnn = {
        'data': [
            go.Heatmap(
                z=metrics_cnn['confusion_matrix'],
                x=['Pred 0', 'Pred 1'],
                y=['True 0', 'True 1'],
                colorscale=custom_colorscale,
                text=metrics_cnn['confusion_matrix'],
                texttemplate="%{text}",
                textfont={"size": 20},
            )
        ],
        'layout': go.Layout(title='Matriz de Confusión CNN', xaxis_nticks=36)
    }

    figure_confusion_lstm = {
        'data': [
            go.Heatmap(
                z=metrics_lstm['confusion_matrix'],
                x=['Pred 0', 'Pred 1'],
                y=['True 0', 'True 1'],
                colorscale=custom_colorscale,
                text=metrics_lstm['confusion_matrix'],
                texttemplate="%{text}",
                textfont={"size": 20},
            )
        ],
        'layout': go.Layout(title='Matriz de Confusión LSTM', xaxis_nticks=36)
    }

    figure_confusion_gru = {
        'data': [
            go.Heatmap(
                z=metrics_gru['confusion_matrix'],
                x=['Pred 0', 'Pred 1'],
                y=['True 0', 'True 1'],
                colorscale=custom_colorscale,
                text=metrics_gru['confusion_matrix'],
                texttemplate="%{text}",
                textfont={"size": 20},
            )
        ],
        'layout': go.Layout(title='Matriz de Confusión GRU', xaxis_nticks=36)
    }

    return figure_classification, figure_mse, figure_mae, figure_r2, figure_confusion_cnn, figure_confusion_lstm, figure_confusion_gru, figure_roc_auc, figure_training_time

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)  # Cambiar el puerto aquí si es necesario
